In [67]:
from pathlib import Path
import json
import os
from tinydb import TinyDB
import re

current_dir = Path(os.getcwd()).absolute()
results_dir = current_dir.joinpath('results')
kv_data_dir = results_dir.joinpath('kvdb')
kv_data_dir.mkdir(parents=True, exist_ok=True)

def _load_json(json_path):
    with open(json_path) as f:
        return json.load(f)

class DocumentDB(object):
    def __init__(self, db_path):
        ## You can use the code from the previous exmaple if you would like
        people_json = kv_data_dir.joinpath('people.json')
        visited_json = kv_data_dir.joinpath('visited.json')
        sites_json = kv_data_dir.joinpath('sites.json')
        measurements_json = kv_data_dir.joinpath('measurements.json')

        self._db_path = Path(db_path)
        self._db = None
        ## TODO: Implement code
        #load json files
        self._person_lookup = _load_json(people_json) #laod people json file
        self._measurements_lookup = _load_json(measurements_json) #laod measurments json file
        self._site_lookup = _load_json(sites_json) #laod site json file
        self._visit_lookup = _load_json(visited_json) #laod visit json file
        
        self._load_db()
        
    def _site_lookup(self, site_id):
        site = []
        #loop through site values
        for value in self._site_lookup.values():
            site.extend(value)
        return site
        
    def _measurements_lookup():  
        return
        
    def _get_site(self, site_id):
        return self._site_lookup[str(site_id)]
    
    def _get_measurements(self, person_id):
        measurements = []
        for values in self._measurements_lookup.values():
            measurements.extend([values for value in values if str(values['person_id']) == str(person_id)]) #changed value to values and value['person_id'] to values['person_id']
        return measurements
    
    def _get_visit(self, visit_id):
        #visit = self._visit_lookup.get(str(visit_id))

        #################################################################
        
        visits = self._visit_lookup #load JSON
        regex = str(visit_id) + r'[^a-zA-Z0-9_]' #create regex to search by visit it
        
        #loop through visits for visit ID matches
        visit = [visits[x] for x in visits if re.search(regex, x)][0] 
        
        ########################################################################
        
        ## TODO: site_id = ...
        site_id = str(visit['site_id'])
        ## TODO: site = ...
        site = self._get_site(site_id)
        visit['site'] = site
        return visit    

    def _load_db(self):
        self._db = TinyDB(self._db_path)
        ## TODO: Implement code
        persons = self._person_lookup.items()
        for person_id, record in persons:
            measurements = self._get_measurements(person_id)          
            visit_ids = set([measurement['visit_id'] for measurement in measurements])      
            visits = []
            for visit_id in visit_ids:       
                visit = self._get_visit(str(visit_id)) #changed self._get_visit(visit_id) to self._get_visit(str(visit_id)) 
                visit['measurements'] = [
                    measurement for measurement in measurements
                    if visit_id == measurement['visit_id']
                ]
                visits.append(visit)
            
            record['visits'] = visits
            self._db.insert(record)   

In [68]:
db_path = results_dir.joinpath('patient-info.json')
if db_path.exists():
    os.remove(db_path)

db = DocumentDB(db_path)